# --- Day 20: Trench Map ---

In [1]:
from Quiz import *
from copy import deepcopy
import numpy as np

### --- Part One ---

With the scanners fully deployed, you turn their attention to mapping the floor of the ocean trench.

When you get back the image from the scanners, it seems to just be random noise. Perhaps you can combine an image enhancement algorithm and the input image (your puzzle input) to clean it up a little.

For example:

`..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..##
#..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###
.######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#.
.#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#.....
.#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#..
...####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.....
..##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#`

`#..#.
#....
##..#
..#..
..###`

The first section is the __image enhancement algorithm__. It is normally given on a single line, but it has been wrapped to multiple lines in this example for legibility. The second section is the __input image__, a two-dimensional grid of __light pixels__ (`#`) and __dark pixels__ (`.`).

The image enhancement algorithm describes how to enhance an image by __simultaneously__ converting all pixels in the input image into an output image. Each pixel of the output image is determined by looking at a 3x3 square of pixels centered on the corresponding input image pixel. So, to determine the value of the pixel at (5,10) in the output image, nine pixels from the input image need to be considered: (4,9), (4,10), (4,11), (5,9), (5,10), (5,11), (6,9), (6,10), and (6,11). These nine input pixels are combined into a single binary number that is used as an index in the __image enhancement algorithm__ string.

For example, to determine the output pixel that corresponds to the very middle pixel of the input image, the nine pixels marked by `[...]` would need to be considered:

`# . . # .
#[. . .].
#[# . .]#
.[. # .].
. . # # #`

Starting from the top-left and reading across each row, these pixels are `...`, then `#..`, then `.#.`; combining these forms `...#...#..` By turning dark pixels (`.`) into `0` and light pixels (`#`) into `1`, the binary number `000100010` can be formed, which is `34` in decimal.

The image enhancement algorithm string is exactly 512 characters long, enough to match every possible 9-bit binary number. The first few characters of the string (numbered starting from zero) are as follows:

`0         10        20        30  `__`34`__`    40        50        60        70
|         |         |         |   `__`|`__`     |         |         |         |
..#.#..#####.#.#.#.###.##.....###.`__`#`__`#.#..###.####..#####..#....#..#..##..##`

In the middle of this first group of characters, the character at index 34 can be found: `#`. So, the output pixel in the center of the output image should be `#`, a __light pixel__.

This process can then be repeated to calculate every pixel of the output image.

Through advances in imaging technology, the images being operated on here are __infinite__ in size. __Every__ pixel of the infinite output image needs to be calculated exactly based on the relevant pixels of the input image. The small input image you have is only a small region of the actual infinite input image; the rest of the input image consists of dark pixels (`.`). For the purposes of the example, to save on space, only a portion of the infinite-sized input and output images will be shown.

The starting input image, therefore, looks something like this, with more dark pixels (`.`) extending forever in every direction not shown here:

`...............
...............
...............
...............
...............
.....#..#......
.....#.........
.....##..#.....
.......#.......
.......###.....
...............
...............
...............
...............
...............`

By applying the image enhancement algorithm to every pixel simultaneously, the following output image can be obtained:

`...............
...............
...............
...............
.....##.##.....
....#..#.#.....
....##.#..#....
....####..#....
.....#..##.....
......##..#....
.......#.#.....
...............
...............
...............
...............`

Through further advances in imaging technology, the above output image can also be used as an input image! This allows it to be enhanced __a second time__:

`...............
...............
...............
..........#....
....#..#.#.....
...#.#...###...
...#...##.#....
...#.....#.#...
....#.#####....
.....#.#####...
......##.##....
.......###.....
...............
...............
...............`

Truly incredible - now the small details are really starting to come through. After enhancing the original input image twice, __`35`__ pixels are lit.

Start with the original input image and apply the image enhancement algorithm twice, being careful to account for the infinite size of the images. 

__How many pixels are lit in the resulting image?__

In [2]:
def prepare(image , algo):
    
    for l in range(len(image)):

        image[l] = list(image[l])

    for l in range(len(image)):

        for c in range(len(image[l])):

            if(image[l][c] == '.'):

                image[l][c] = '0'

            else:

                image[l][c] = '1'
          
    for i in range(len(algo)):

        if(algo[i] == '.'):

            algo[i] = '0'

        else:

            algo[i] = '1'

    return (image , algo)

In [3]:
def enhance(img , num_of_steps = 2 , fill_value = 0 , pad_size = 2):
    
    for steps in range(num_of_steps):
        
        img = np.pad(img , pad_size , constant_values = fill_value)
        rows = img.shape[0]
        cols = img.shape[1]
        
        stride_rows = rows - pad_size
        stride_cols = cols - pad_size
        stride_shape = (stride_rows , stride_cols , 3 , 3)
        
        stride = np.lib.stride_tricks.as_strided(img , stride_shape , 2 * img.strides)
        stride = np.reshape(stride , (stride_rows , stride_cols , 9))
        
        codes = stride[: , : , 0] * 256 + np.packbits(stride[: , : , 1:]).reshape(stride_rows , stride_cols)
        
        img = algorithm[codes]
        
        fill_value = algorithm[fill_value]
        
    return img

#### Test

In [4]:
image = deepcopy(test_input_image)
algo = list(deepcopy(test_input_algo))

prep = prepare(image , algo)
img = prep[0]
algorithm = prep[1]

algorithm = np.array([int(x) for x in algorithm])
img_temp = []

for row in img:
    
    img_temp.append([int(x) for x in row])

img_in = np.array(img_temp)

print("Lit pixels after 2 enhancements:" , np.sum(enhance(img_in)))

Lit pixels after 2 enhancements: 35


#### Answer

In [5]:
image = deepcopy(input_image)
algo = list(deepcopy(input_algo))

prep = prepare(image , algo)
img = prep[0]
algorithm = prep[1]

algorithm = np.array([int(x) for x in algorithm])
img_temp = []

for row in img:
    
    img_temp.append([int(x) for x in row])

img_in = np.array(img_temp)

print("Lit pixels after 2 enhancements:" , np.sum(enhance(img_in)))

Lit pixels after 2 enhancements: 4928


-------------------------------------

### --- Part Two ---

You still can't quite make out the details in the image. Maybe you just didn't enhance https://en.wikipedia.org/wiki/Kernel_(image_processing) it <ins>enough</ins>.

If you enhance the starting input image in the above example a total of __`50`__ times, __`3351`__ pixels are lit in the final output image.

Start again with the original input image and apply the image enhancement algorithm 50 times.

__How many pixels are lit in the resulting image?__

#### Test

In [6]:
image = deepcopy(test_input_image)
algo = list(deepcopy(test_input_algo))

prep = prepare(image , algo)
img = prep[0]
algorithm = prep[1]

algorithm = np.array([int(x) for x in algorithm])
img_temp = []

for row in img:
    
    img_temp.append([int(x) for x in row])

img_in = np.array(img_temp)

print("Lit pixels after 50 enhancements:" , np.sum(enhance(img_in , 50)))

Lit pixels after 50 enhancements: 3351


#### Answer

In [7]:
image = deepcopy(input_image)
algo = list(deepcopy(input_algo))

prep = prepare(image , algo)
img = prep[0]
algorithm = prep[1]

algorithm = np.array([int(x) for x in algorithm])
img_temp = []

for row in img:
    
    img_temp.append([int(x) for x in row])

img_in = np.array(img_temp)

print("Lit pixels after 50 enhancements:" , np.sum(enhance(img_in , 50)))

Lit pixels after 50 enhancements: 16605
